In [77]:
!pip install streamlit pandas # Install Streamlit and pandas
!pip install pyngrok # For ngrok tunnel, which is generally more reliable

In [78]:
!ngrok authtoken 2yxVXCrmtPGi226dvGWcPMHD8MI_73k2EmZ9zJ7io8xidgroS

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [79]:
%%writefile app.py

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import streamlit as st
import seaborn as sns
import numpy as np # Import numpy for isnan
import re # Import the regular expression module
from io import BytesIO # To read uploaded files

# Hardcoded Regulatory/Industry Thresholds and Scoring Logic (as discussed)
# For simplicity, using a 5-point scale: 5 (Excellent) to 1 (Critical)
# Bands are exclusive on the upper bound: [lower, upper)

metric_info = {
    "Cash Ratio": {
        "description": "Indicates a bank's ability to cover its short-term liabilities with its most liquid assets. Often superseded by LCR for regulatory purposes.",
        "type": "higher_better",
        "benchmark": "LCR >= 100%", # As per Basel III Liquidity Coverage Ratio
        "bands": {
            (120, float('inf')): 5, # Well above LCR equivalent
            (100, 120): 4,          # At/above LCR equivalent
            (80, 100): 3,           # Near/below (watch)
            (60, 80): 2,            # Concerning
            (0, 60): 1              # Critical
        }
    },
    "Cost-to-Income Ratio": {
        "description": "Measures a bank's operational efficiency. Lower is generally better.",
        "type": "lower_better",
        "benchmark": "< 60%", # Industry benchmark
        "bands": {
            (0, 45): 5,
            (45, 55): 4,
            (55, 65): 3,
            (65, 75): 2,
            (75, float('inf')): 1
        }
    },
    "Current Ratio": {
        "description": "A general liquidity ratio, current assets to current liabilities. For banks, LCR/NSFR are more specific.",
        "type": "higher_better",
        "benchmark": "> 1 (general corporate, for banks it's about robust liquidity management via LCR/NSFR)",
        "bands": {
            (2, float('inf')): 5,
            (1.5, 2): 4,
            (1, 1.5): 3,
            (0.5, 1): 2,
            (0, 0.5): 1
        }
    },
    "Debt-to-Equity Ratio": {
        "description": "Measures a bank's leverage. Banks typically have higher ratios due to deposits. Regulators focus on capital ratios.",
        "type": "lower_better",
        "benchmark": "No explicit regulatory max, but lower D/E implies less leverage.",
        "bands": { # Adjusted for typical bank operations, where higher D/E is normal
            (0, 15): 5, # Very strong capital base relative to debt
            (15, 25): 4,
            (25, 35): 3,
            (35, 45): 2,
            (45, float('inf')): 1
        }
    },
    "Equity-to-Assets Ratio": {
        "description": "Measures the proportion of assets financed by equity. Inverse of the leverage ratio (assets/equity).",
        "type": "higher_better",
        "benchmark": "Related to Leverage Ratio (Tier 1 Capital / Total Assets) >= 3% (Basel III)",
        "bands": { # Mapping to the 3% Basel III Leverage Ratio expectation
            (0.05, float('inf')): 5, # > 5% Equity/Assets
            (0.04, 0.05): 4,         # 4-5% Equity/Assets
            (0.03, 0.04): 3,         # 3-4% Equity/Assets (at/above benchmark)
            (0.02, 0.03): 2,         # 2-3% Equity/Assets (below benchmark)
            (0, 0.02): 1             # < 2% Equity/Assets (critical)
        }
    },
    "Net Interest Margin (NIM)": {
        "description": "A key profitability measure, showing the difference between interest income and interest expense, relative to earning assets.",
        "type": "higher_better",
        "benchmark": "No regulatory min; generally 2-4% is considered healthy.",
        "bands": {
            (0.04, float('inf')): 5, # > 4%
            (0.03, 0.04): 4,         # 3-4%
            (0.02, 0.03): 3,         # 2-3%
            (0.01, 0.02): 2,         # 1-2%
            (float('-inf'), 0.01): 1 # < 1% or negative
        }
    },
    "Non-Performing Loan (NPL) Ratio": {
        "description": "Measures asset quality and credit risk. Lower is better.",
        "type": "lower_better",
        "benchmark": "< 5% (often aims for < 2-3% in healthy economies)",
        "bands": {
            (0, 0.01): 5,            # < 1%
            (0.01, 0.03): 4,         # 1-3%
            (0.03, 0.05): 3,         # 3-5%
            (0.05, 0.07): 2,         # 5-7%
            (0.07, float('inf')): 1  # > 7%
        }
    },
    "Return on Assets (ROA)": {
        "description": "Measures how efficiently a bank uses its assets to generate profit.",
        "type": "higher_better",
        "benchmark": "No regulatory min; generally > 0.5% for healthy banks.",
        "bands": {
            (0.01, float('inf')): 5, # > 1%
            (0.0075, 0.01): 4,       # 0.75-1%
            (0.005, 0.0075): 3,      # 0.5-0.75%
            (0, 0.005): 2,           # 0-0.5%
            (float('-inf'), 0): 1    # Negative
        }
    },
    "Return on Equity (ROE)": {
        "description": "Measures the return generated on shareholders' equity. High ROE can also signal high leverage.",
        "type": "higher_better",
        "benchmark": "No regulatory min; generally > 8-10% for healthy banks.",
        "bands": {
            (0.15, float('inf')): 5, # > 15%
            (0.10, 0.15): 4,         # 10-15%
            (0.08, 0.10): 3,         # 8-10%
            (0, 0.08): 2,            # 0-8%
            (float('-inf'), 0): 1    # Negative
        }
    },
    "Tier 1 Capital Ratio": {
        "description": "A core measure of a bank's financial strength and ability to absorb losses.",
        "type": "higher_better",
        "benchmark": ">= 6% (Basel III minimum for Tier 1 Capital)",
        "bands": {
            (0.08, float('inf')): 5, # > 8% (well-capitalized in many contexts)
            (0.07, 0.08): 4,         # 7-8%
            (0.06, 0.07): 3,         # 6-7% (at/above regulatory minimum)
            (0.045, 0.06): 2,        # 4.5-6% (under-capitalized warning)
            (0, 0.045): 1            # < 4.5% (significantly under-capitalized)
        }
    }
}

def get_metric_score(metric_name, value):
    """Calculates a score (1-5) for a given metric value based on defined bands."""
    info = metric_info.get(metric_name)
    if not info or pd.isna(value):
        return None

    # Handle percentage values for certain metrics (e.g., if input is 56.48 for 56.48%)
    # Convert to decimal for consistent comparison with `bands` if bands are in decimal
    # The original data for ratios like Equity-to-Assets, NIM, NPL, ROA, ROE, Tier 1 are assumed decimals.
    # Cost-to-Income and Cash Ratio bands are defined expecting potentially higher integer values.
    # No general division by 100 here, comparison is direct.

    for (lower, upper), score in info["bands"].items():
        if info["type"] == "higher_better":
            # Check for lower bound inclusivity and upper bound exclusivity
            if (lower is None or value >= lower) and (upper is None or value < upper):
                 return score
        elif info["type"] == "lower_better":
             # Check for lower bound exclusivity and upper bound inclusivity
             if (lower is None or value > lower) and (upper is None or value <= upper):
                 return score
    return None


def calculate_quarterly_health(df_quarter):
    """Calculates an overall weighted health score for a given quarter."""
    total_weighted_score = 0
    total_weight = 0
    detailed_scores = {}

    # Define example weights for metrics (can be adjusted)
    metric_weights = {
        "Tier 1 Capital Ratio": 3,
        "Non-Performing Loan (NPL) Ratio": 2.5,
        "Equity-to-Assets Ratio": 2,
        "Cash Ratio": 1.5,
        "Net Interest Margin (NIM)": 1.5,
        "Return on Assets (ROA)": 1,
        "Return on Equity (ROE)": 1,
        "Cost-to-Income Ratio": 0.5,
        "Debt-to-Equity Ratio": 0.5,
        "Current Ratio": 0.5,
    }

    for index, row in df_quarter.iterrows():
        metric_name = row['Financial Metric Name']
        actual_value = row['Actual Value']
        qoq_change = row['% Change (QoQ)']
        weight = metric_weights.get(metric_name, 0)

        if pd.isna(actual_value) or weight == 0:
            detailed_scores[metric_name] = {"Score": "N/A", "Details": "Missing value or no weight assigned."}
            continue

        metric_base_score = get_metric_score(metric_name, actual_value)

        if metric_base_score is not None:
            change_adjustment = 0
            if not pd.isna(qoq_change): # Only apply adjustment if QoQ change is available
                # Adjust QoQ change if it's in a non-standard scale (e.g. a factor instead of percent)
                # Assuming QoQ change is already a percentage (e.g., 10 for 10% increase)
                if qoq_change > 10 and metric_info[metric_name]["type"] == "higher_better":
                    change_adjustment = 1
                elif qoq_change < -10 and metric_info[metric_name]["type"] == "lower_better":
                    change_adjustment = 1
                elif qoq_change < -10 and metric_info[metric_name]["type"] == "higher_better":
                    change_adjustment = -1
                elif qoq_change > 10 and metric_info[metric_name]["type"] == "lower_better":
                    change_adjustment = -1

            final_metric_score = max(1, min(5, metric_base_score + change_adjustment))
            total_weighted_score += (final_metric_score * weight)
            total_weight += weight
            detailed_scores[metric_name] = {
                "Actual Value": f"{actual_value:.4f}" if abs(actual_value) < 1 and actual_value != 0 else f"{actual_value:.2f}",
                "QoQ Change (%)": f"{qoq_change:.2f}" if not pd.isna(qoq_change) else "N/A",
                "Base Score": metric_base_score,
                "Adjusted Score": final_metric_score,
                "Details": f"Weighted Score: {final_metric_score * weight:.2f}"
            }
        else:
            detailed_scores[metric_name] = {"Score": "N/A", "Details": f"Could not score value {actual_value:.2f}"}

    overall_average_score = total_weighted_score / total_weight if total_weight > 0 else 0
    return overall_average_score, detailed_scores

def process_uploaded_file(uploaded_file):
    """Reads an uploaded Excel file and prepares the DataFrame."""
    try:
        # Read the Excel file into a pandas DataFrame
        df = pd.read_excel(uploaded_file)

        # Ensure required columns are present
        required_cols = ['Bank Name', 'Year', 'Quarter', 'Financial Metric Name', 'Actual Value', '% Change (QoQ)']
        if not all(col in df.columns for col in required_cols):
            st.error(f"Uploaded file is missing required columns. Please ensure it contains: {', '.join(required_cols)}")
            return None

        # Create YearQuarter column
        df['YearQuarter'] = df['Year'].astype(str) + ' Q' + df['Quarter'].astype(str)

        return df
    except Exception as e:
        st.error(f"Error processing file {uploaded_file.name}: {e}")
        return None

# --- Streamlit UI ---
st.set_page_config(layout="wide", page_title="Bank Fiscal Health Dashboard")

st.title("Multi-Bank Fiscal Health Dashboard (QoQ Assessment)")
st.markdown("Upload Excel files for different banks to compare their fiscal health.")

# File Uploader
uploaded_files = st.file_uploader("Upload one or more Bank Fiscal Health Excel files (.xlsx)", type="xlsx", accept_multiple_files=True)

bank_data = {} # Dictionary to store DataFrame for each bank

if uploaded_files:
    for file in uploaded_files:
        # Use the first 3 letters of the filename (without extension) as the bank name
        bank_name = file.name.rsplit('.', 1)[0][:3].upper() # Take first 3 chars and make uppercase
        st.sidebar.info(f"Processing {file.name} as **{bank_name}**")
        df_bank = process_uploaded_file(file)
        if df_bank is not None:
            # Ensure 'Bank Name' column in the DataFrame is consistent
            df_bank['Bank Name'] = bank_name
            bank_data[bank_name] = df_bank
        else:
            st.sidebar.warning(f"Skipping {file.name} due to processing error.")

if bank_data:
    # Get all unique quarters across all uploaded files
    all_quarters = sorted(list(set(quarter for df in bank_data.values() for quarter in df['YearQuarter'].unique())),
                          key=lambda x: (int(x.split(' Q')[0]), int(x.split(' Q')[1])))

    if not all_quarters:
        st.warning("No valid quarter data found in the uploaded files.")
    else:
        # Create tabs for each bank and a final comparison tab
        tab_titles = list(bank_data.keys()) + ["Overall Comparison"]
        tabs = st.tabs(tab_titles)

        # --- Individual Bank Tabs ---
        for i, bank_name in enumerate(bank_data.keys()):
            with tabs[i]:
                st.header(f"Analysis for {bank_name}")

                df_bank = bank_data[bank_name]

                # Quarter selection specific to this bank's data
                bank_quarters = sorted(df_bank['YearQuarter'].unique(),
                                       key=lambda x: (int(x.split(' Q')[0]), int(x.split(' Q')[1])))

                if not bank_quarters:
                    st.info("No quarter data available for this bank.")
                    continue

                selected_quarter = st.selectbox(f"Select Quarter for {bank_name}:", bank_quarters,
                                                 index=len(bank_quarters)-1, key=f"{bank_name}_quarter_select")

                df_selected_quarter = df_bank[df_bank['YearQuarter'] == selected_quarter].copy()

                # Calculate and display overall health score for the selected quarter
                overall_score, detailed_scores = calculate_quarterly_health(df_selected_quarter)

                st.markdown(f"### Overall Quantitatively Derived Health Score (Avg. 1-5): **{overall_score:.2f}**")

                if overall_score >= 4.5:
                    st.success("Overall Health: Excellent! (Strong performance across metrics)")
                elif overall_score >= 3.5:
                    st.info("Overall Health: Good (Generally healthy, meeting most benchmarks)")
                elif overall_score >= 2.5:
                    st.warning("Overall Health: Neutral/Watch (Some areas of concern, requires monitoring)")
                else:
                    st.error("Overall Health: Concerning/Critical (Significant distress signals)")

                st.markdown("---")
                st.subheader("Detailed Metric Analysis")

                # Display metrics using st.container for better grouping and borders
                cols = st.columns(3) # Display metrics in columns

                j = 0
                for metric_name in metric_info.keys():
                    metric_data_available = not df_selected_quarter[df_selected_quarter['Financial Metric Name'] == metric_name].empty

                    if metric_data_available:
                         current_col = cols[j % 3]
                         with current_col:
                             with st.container(border=True): # Use container with border
                                 st.markdown(f"#### {metric_name}")
                                 st.write(f"**Description:** {metric_info[metric_name]['description']}")
                                 st.write(f"**Benchmark/Regulatory Target:** {metric_info[metric_name]['benchmark']}")

                                 metric_row = df_selected_quarter[df_selected_quarter['Financial Metric Name'] == metric_name].iloc[0]
                                 actual_value = metric_row['Actual Value']
                                 qoq_change = metric_row['% Change (QoQ)']

                                 score_details = detailed_scores.get(metric_name, {})

                                 if score_details.get("Score") == "N/A":
                                      st.write(f"**Actual Value:** N/A")
                                      st.write(f"**QoQ Change:** N/A")
                                      st.write(f"**Score:** N/A (Details: {score_details.get('Details')})")
                                 else:
                                      # Special handling for percentage-based values to display them clearly
                                      display_value = actual_value
                                      if metric_name in ["Tier 1 Capital Ratio", "Non-Performing Loan (NPL) Ratio", "Equity-to-Assets Ratio",
                                                        "Net Interest Margin (NIM)", "Return on Assets (ROA)", "Return on Equity (ROE)"]:
                                          if not pd.isna(actual_value) and actual_value < 0.2 and actual_value != 0:
                                              display_value = actual_value * 100 # Convert to percentage for display
                                              st.write(f"**Actual Value:** {display_value:.2f}%")
                                          else:
                                              st.write(f"**Actual Value:** {display_value:.2f}")
                                      else:
                                           if not pd.isna(actual_value):
                                              st.write(f"**Actual Value:** {display_value:.2f}")
                                           else:
                                              st.write(f"**Actual Value:** N/A")


                                      st.write(f"**QoQ Change:** {qoq_change:.2f}%" if not pd.isna(qoq_change) else "**QoQ Change:** N/A")
                                      st.write(f"**Derived Score (1-5):** {score_details.get('Adjusted Score')}")

                         j += 1

                st.markdown("---")
                st.subheader("Trend Analysis Across Quarters")

                # Create a DataFrame suitable for plotting trends for this bank
                df_pivot = df_bank.pivot_table(
                    index='YearQuarter',
                    columns='Financial Metric Name',
                    values='Actual Value'
                ).reindex(bank_quarters) # Ensure correct quarter order

                # Identify metrics with data across quarters for this bank
                metrics_with_data = [col for col in df_pivot.columns if df_pivot[col].notna().sum() > 0]

                num_metrics_with_data = len(metrics_with_data)

                if num_metrics_with_data > 0:
                    # Adjust figure size based on the number of metrics to avoid overcrowding
                    nrows = (num_metrics_with_data + 2) // 3
                    fig, axes = plt.subplots(nrows=nrows, ncols=3, figsize=(18, 5 * nrows))
                    axes = axes.flatten() # Flatten to easily iterate

                    for k, metric_name in enumerate(metrics_with_data):
                        ax = axes[k]
                        df_pivot[metric_name].plot(ax=ax, marker='o')
                        ax.set_title(metric_name)
                        ax.set_xlabel("Quarter")
                        ax.set_ylabel("Value")
                        ax.tick_params(axis='x', rotation=45)
                        ax.grid(True, linestyle='--', alpha=0.6)

                        # Add benchmark line if available and numeric
                        benchmark_str = metric_info.get(metric_name, {}).get('benchmark')
                        if benchmark_str and any(char.isdigit() for char in benchmark_str):
                            try:
                                # Attempt to extract a numeric value from the benchmark string
                                benchmark_value_match = re.search(r'(\d+\.?\d*)', benchmark_str)
                                if benchmark_value_match:
                                    benchmark_value = float(benchmark_value_match.group(1))
                                    # Adjust benchmark if it's given as % but plotted data is decimal (and vice-versa)
                                    if metric_name in ["Tier 1 Capital Ratio", "NPL Ratio", "ROA", "ROE", "Net Interest Margin (NIM)", "Equity-to-Assets Ratio"] and benchmark_value > 1:
                                         benchmark_value /= 100.0
                                    elif metric_name in ["Cost-to-Income Ratio"] and benchmark_value < 1:
                                         benchmark_value *= 100.0

                                    ax.axhline(benchmark_value, color='r', linestyle='--', label=f'Benchmark (~{benchmark_value:.2f})')
                                    ax.legend()
                            except (ValueError, AttributeError):
                                pass # Ignore if benchmark string is complex or doesn't contain easily parsable number

                    # Hide any unused subplots
                    for l in range(k + 1, len(axes)):
                        fig.delaxes(axes[l])

                    plt.tight_layout()
                    st.pyplot(fig)
                else:
                    st.info("No trend data available for the selected metrics across quarters for this bank.")


        # --- Overall Comparison Tab ---
        with tabs[len(bank_data)]:
            st.header("Overall Comparison Across Banks")

            if not all_quarters:
                st.info("No quarter data available for comparison.")
            else:
                # Select quarter for comparison
                selected_comparison_quarter = st.selectbox("Select Quarter for Comparison:", all_quarters,
                                                           index=len(all_quarters)-1, key="overall_comparison_quarter")

                comparison_scores = {}
                for bank_name, df_bank in bank_data.items():
                    df_quarter = df_bank[df_bank['YearQuarter'] == selected_comparison_quarter].copy()
                    if not df_quarter.empty:
                         overall_score, _ = calculate_quarterly_health(df_quarter)
                         comparison_scores[bank_name] = overall_score
                    # Do not add banks with no data for the selected quarter to the comparison
                    # else:
                    #      comparison_scores[bank_name] = 0 # Or np.nan, or exclude? Using 0 for now.


                if comparison_scores:
                    # Create a DataFrame for comparison
                    df_comparison = pd.DataFrame.from_dict(comparison_scores, orient='index', columns=['Overall Health Score'])
                    df_comparison = df_comparison.sort_values(by='Overall Health Score', ascending=False)

                    st.subheader(f"Overall Health Scores for {selected_comparison_quarter}")
                    st.dataframe(df_comparison)

                    # Plot comparison scores
                    fig_comp, ax_comp = plt.subplots(figsize=(12, 6))
                    sns.barplot(x=df_comparison.index, y='Overall Health Score', data=df_comparison, ax=ax_comp)
                    ax_comp.set_title(f'Overall Health Score Comparison - {selected_comparison_quarter}')
                    ax_comp.set_ylabel('Overall Health Score (Avg. 1-5)')
                    ax_comp.set_xlabel('Bank Name')
                    ax_comp.set_ylim(0, 5) # Set y-axis limits to the score range
                    plt.xticks(rotation=45, ha='right')
                    plt.tight_layout()
                    st.pyplot(fig_comp)

                else:
                    st.info("No data available for comparison in the selected quarter for any uploaded bank.")

            st.markdown("---")
            st.subheader("Quarterly Health Score Trends Across Banks")

            # Calculate health scores for all banks across all quarters
            all_banks_scores = []
            for bank_name, df_bank in bank_data.items():
                bank_quarters = sorted(df_bank['YearQuarter'].unique(),
                                      key=lambda x: (int(x.split(' Q')[0]), int(x.split(' Q')[1])))
                for quarter in bank_quarters:
                    df_quarter = df_bank[df_bank['YearQuarter'] == quarter].copy()
                    # Only calculate score if there's data for the quarter for this bank
                    if not df_quarter.empty:
                        overall_score, _ = calculate_quarterly_health(df_quarter)
                        all_banks_scores.append({'Bank Name': bank_name, 'YearQuarter': quarter, 'Health Score': overall_score})

            if all_banks_scores:
                df_all_scores = pd.DataFrame(all_banks_scores)

                # Pivot for plotting
                df_all_scores_pivot = df_all_scores.pivot(index='YearQuarter', columns='Bank Name', values='Health Score').reindex(all_quarters)

                st.subheader("Health Score Trend Over Time")
                fig_trend, ax_trend = plt.subplots(figsize=(14, 7))

                # Only plot columns (banks) that have data
                banks_with_trend_data = [col for col in df_all_scores_pivot.columns if df_all_scores_pivot[col].notna().sum() > 0]

                if banks_with_trend_data:
                    for bank_name in banks_with_trend_data:
                        df_all_scores_pivot[bank_name].plot(ax=ax_trend, marker='o', label=bank_name)

                    ax_trend.set_title('Quarterly Health Score Trend Across Banks')
                    ax_trend.set_xlabel('Quarter')
                    ax_trend.set_ylabel('Overall Health Score (Avg. 1-5)')
                    ax_trend.set_ylim(0, 5.5) # Add a bit of padding
                    ax_trend.tick_params(axis='x', rotation=45)
                    ax_trend.grid(True, linestyle='--', alpha=0.6)
                    ax_trend.legend(title='Bank')
                    plt.tight_layout()
                    st.pyplot(fig_trend)
                else:
                    st.info("Not enough data to show quarterly trends across banks.")


            else:
                st.info("Not enough data to show quarterly trends across banks.")


Overwriting app.py


In [80]:
from pyngrok import ngrok
import os

# Terminate any existing ngrok tunnels if you restart the cell
ngrok.kill()

# Use the authtoken from environment variable if set, or set it directly
# os.environ["NGROK_AUTH_TOKEN"] = "YOUR_NGROK_AUTH_TOKEN" # Uncomment and set if you didn't run Step 3

# Connect to ngrok (Streamlit typically runs on port 8501)
public_url = ngrok.connect(addr="8501")
print(f"Streamlit App URL: {public_url}")

# Run Streamlit in the background
!streamlit run app.py --server.port 8501 --server.enableCORS false --server.enableXsrfProtection false &

Streamlit App URL: NgrokTunnel: "https://39b4-35-196-48-212.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.196.48.212:8501

  Stopping...
